In [1]:
import cdsapi
import xarray as xr
import numpy as np
import os
import time
from pathlib import Path
import glob

    
def download_era5_year(year, output_dir="data/era5/yearly"):
    
    year_str = str(year)
    print(f"Downloading ERA5 data for year {year_str}...")
    
    os.makedirs(output_dir, exist_ok=True)
    
    output_file = os.path.join(output_dir, f"era5_temp_{year_str}.nc")
    if os.path.exists(output_file):
        print(f"File for {year_str} already exists: {output_file}")
        return output_file
    
    dataset = "reanalysis-era5-single-levels"
    request = {
        "product_type": ["reanalysis"],
        "variable": ["2m_temperature"],
        "year": [year_str],
        "month": [f"{i:02d}" for i in range(1, 13)],
        "day": [f"{i:02d}" for i in range(1, 32)],
        "time": ["12:00"],
        "data_format": "netcdf",
        "download_format": "unarchived"
    }
    
    client = cdsapi.Client()
    
    try:
        result = client.retrieve(dataset, request)
        result.download(output_file)
        
        if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
            size_mb = os.path.getsize(output_file) / 1e6
            print(f"Downloaded: {output_file} ({size_mb:.1f} MB)")
            return output_file
        else:
            print(f"File not created or empty: {output_file}")
            if os.path.exists(output_file):
                os.remove(output_file) 
            return None
            
    except Exception as e:
        print(f"Download failed for {year_str}: {e}")
        return None
    
    


def download_era5_batch(start_year, end_year, output_dir="data/era5/yearly", delay_between_downloads=60):
    
    years = list(range(start_year, end_year + 1))
    total_years = len(years)
    
    successful_downloads = []
    failed_downloads = []
    
    for i, year in enumerate(years, 1):
        
        # Check if file already exists before calling download function
        year_str = str(year)
        output_file = os.path.join(output_dir, f"era5_temp_{year_str}.nc")
        file_already_exists = os.path.exists(output_file)
        
        result = download_era5_year(year, output_dir)
        
        if result:
            successful_downloads.append(year)
            print(f"Success: {year}")
            
            # Only delay if file was actually downloaded (not if it already existed)
            if not file_already_exists and i < total_years:
                print(f"Waiting {delay_between_downloads} seconds before next download...")
                time.sleep(delay_between_downloads)
            elif file_already_exists and i < total_years:
                print("File already existed - no delay needed.")
        else:
            failed_downloads.append(year)
            print(f"Failed: {year}")
    
    print(f"Successful: {len(successful_downloads)} years")
    print(f"Failed: {len(failed_downloads)} years")
    
    return successful_downloads, failed_downloads


def concatenate_yearly_files_to_netcdf(input_dir="data/era5/yearly", output_nc="data/era5/era5_t2m_1940-2014_daily.nc"):
    """
    Concatenate all yearly ERA5 files into a single NetCDF file
    """
    
    os.makedirs(os.path.dirname(output_nc), exist_ok=True)
    
    # Remove existing file if it exists
    if os.path.exists(output_nc):
        print(f"Removing existing NetCDF file: {output_nc}")
        os.remove(output_nc)
    
    file_pattern = os.path.join(input_dir, "era5_temp_*.nc")
    yearly_files = sorted(glob.glob(file_pattern))
    
    if not yearly_files:
        print(f"No files found matching pattern: {file_pattern}")
        return None
    
    print(f"Found {len(yearly_files)} files to process:")
    for f in yearly_files:
        print(f"  - {os.path.basename(f)}")
    
    try:
        print(f"\nConcatenating files...")
        
        # Open all datasets and concatenate along time dimension
        datasets = [xr.open_dataset(f) for f in yearly_files]
        combined_ds = xr.concat(datasets, dim='valid_time')
        
        print(f"Combined dataset info:")
        print(f"  - Shape: {combined_ds.dims}")
        print(f"  - Time range: {combined_ds.valid_time.values[0]} to {combined_ds.valid_time.values[-1]}")
        print(f"  - Variables: {list(combined_ds.data_vars)}")
        
        # Save to NetCDF
        print(f"\nSaving to NetCDF: {output_nc}")
        combined_ds.to_netcdf(output_nc)
        
        # Calculate file size
        nc_size_gb = os.path.getsize(output_nc) / 1e9
        print(f"  - NetCDF file size: {nc_size_gb:.1f} GB")
        
        # Close datasets
        for ds in datasets:
            ds.close()
        combined_ds.close()
        
        print(f"\nNetCDF file created successfully!")
        return output_nc
        
    except Exception as e:
        print(f"NetCDF concatenation failed: {e}")
        # Clean up partial file on failure
        if os.path.exists(output_nc):
            os.remove(output_nc)
        return None

    
    
def create_era5_full_dataset(start_year=1940, end_year=2014, download_delay=60):
    """
    Download yearly and concatenate ERA5 data to NetCDF
    
    Returns:
        xarray.Dataset: Combined ERA5 dataset
    """

    successful, failed = download_era5_batch(start_year, end_year, delay_between_downloads=download_delay)
    
    if not successful:  
        print("No files available. Cannot proceed.")
        return None
    
    if failed:
        print(f"Warning: {len(failed)} years failed: {failed}")
    
    # Create NetCDF file
    nc_path = concatenate_yearly_files_to_netcdf()
    if nc_path is None:
        return None
    
    # Load and return dataset
    try:
        ds = xr.open_dataset(nc_path)
        print(f"Loaded ERA5 dataset: {ds.dims}")
        return ds
    except Exception as e:
        print(f"Failed to load: {e}")
        return None

/home.ufs/yw4236/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home.ufs/yw4236/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ds = create_era5_full_dataset(start_year=1940, end_year=2014, download_delay=5)

File for 1940 already exists: data/era5/yearly/era5_temp_1940.nc
Success: 1940
File already existed - no delay needed.
File for 1941 already exists: data/era5/yearly/era5_temp_1941.nc
Success: 1941
File already existed - no delay needed.
File for 1942 already exists: data/era5/yearly/era5_temp_1942.nc
Success: 1942
File already existed - no delay needed.
File for 1943 already exists: data/era5/yearly/era5_temp_1943.nc
Success: 1943
File already existed - no delay needed.
File for 1944 already exists: data/era5/yearly/era5_temp_1944.nc
Success: 1944
File already existed - no delay needed.
File for 1945 already exists: data/era5/yearly/era5_temp_1945.nc
Success: 1945
File already existed - no delay needed.
File for 1946 already exists: data/era5/yearly/era5_temp_1946.nc
Success: 1946
File already existed - no delay needed.
File for 1947 already exists: data/era5/yearly/era5_temp_1947.nc
Success: 1947
File already existed - no delay needed.
File for 1948 already exists: data/era5/yearly/e

2026-01-13 21:28:08,123 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2026-01-13 21:28:08,124 INFO Request ID is d92de665-01fe-44fd-8af2-aad59bdac229
2026-01-13 21:28:08,708 INFO status has been updated to accepted
2026-01-13 21:28:41,995 INFO status has been updated to successful


7084c40dff197b95685cfbfc85cbc973.nc:   0%|          | 0.00/521M [00:00<?, ?B/s]

Downloaded: data/era5/yearly/era5_temp_1960.nc (546.3 MB)
Success: 1960
Waiting 5 seconds before next download...


2026-01-13 21:29:14,205 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2026-01-13 21:29:14,207 INFO Request ID is 366657b1-7ff3-4b03-921d-495b27fa52b3
2026-01-13 21:29:14,566 INFO status has been updated to accepted
2026-01-13 21:29:25,162 INFO status has been updated to running
2026-01-13 21:30:39,371 INFO status has been updated to successful


...

8618dae8be785be32b98ed4bc8c46a50.nc:   0%|          | 0.00/521M [00:00<?, ?B/s]

Downloaded: data/era5/yearly/era5_temp_2013.nc (545.8 MB)
Success: 2013
Waiting 5 seconds before next download...


2026-01-13 23:37:54,833 INFO [2025-12-11T00:00:00] Please note that a dedicated catalogue entry for this dataset, post-processed and stored in Analysis Ready Cloud Optimized (ARCO) format (Zarr), is available for optimised time-series retrievals (i.e. for retrieving data from selected variables for a single point over an extended period of time in an efficient way). You can discover it [here](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels-timeseries?tab=overview)
2026-01-13 23:37:54,835 INFO Request ID is 402a5b84-2a3c-4462-8f0f-93994e5fe4ab
2026-01-13 23:37:54,962 INFO status has been updated to accepted
2026-01-13 23:38:16,586 INFO status has been updated to running
2026-01-13 23:39:49,715 INFO status has been updated to successful


84395ba1ea7f03510a8eeb05f98a366b.nc:   0%|          | 0.00/521M [00:00<?, ?B/s]

Downloaded: data/era5/yearly/era5_temp_2014.nc (546.2 MB)
Success: 2014
Successful: 75 years
Failed: 0 years
Found 75 files to process:
  - era5_temp_1940.nc
  - era5_temp_1941.nc
  - era5_temp_1942.nc
  - era5_temp_1943.nc
  - era5_temp_1944.nc
  - era5_temp_1945.nc
  - era5_temp_1946.nc
  - era5_temp_1947.nc
  - era5_temp_1948.nc
  - era5_temp_1949.nc
  - era5_temp_1950.nc
  - era5_temp_1951.nc
  - era5_temp_1952.nc
  - era5_temp_1953.nc
  - era5_temp_1954.nc
  - era5_temp_1955.nc
  - era5_temp_1956.nc
  - era5_temp_1957.nc
  - era5_temp_1958.nc
  - era5_temp_1959.nc
  - era5_temp_1960.nc
  - era5_temp_1961.nc
  - era5_temp_1962.nc
  - era5_temp_1963.nc
  - era5_temp_1964.nc
  - era5_temp_1965.nc
  - era5_temp_1966.nc
  - era5_temp_1967.nc
  - era5_temp_1968.nc
  - era5_temp_1969.nc
  - era5_temp_1970.nc
  - era5_temp_1971.nc
  - era5_temp_1972.nc
  - era5_temp_1973.nc
  - era5_temp_1974.nc
  - era5_temp_1975.nc
  - era5_temp_1976.nc
  - era5_temp_1977.nc
  - era5_temp_1978.nc
  - er